## 1

In [3]:
from functools import reduce
import math

def map_with_reduce(fn, iterable):
    return reduce(lambda acc, x: acc + [fn(x)], iterable, [])


In [4]:
map_with_reduce(lambda x: math.exp(math.sin(x)), [1, 2, 3, 4, 5, 6, 7])

[2.319776824715853,
 2.4825777280150008,
 1.151562836514535,
 0.46916418587400077,
 0.3833049951722714,
 0.7562256275428552,
 1.9289708044108762]

In [7]:
def filter_with_reduce(fn, iterable):
    return reduce(lambda acc, x: acc + ([x] if fn(x) else []), iterable, [])

In [8]:
filter_with_reduce(lambda x: int(x) % 2 == 0, [0, 1, 2, 3, 4, 5, 5.5, 6.4])

[0, 2, 4, 6.4]

In [9]:
def compose_with_reduce(iterable):
    return reduce(lambda acc, x: lambda el: acc(x(el)), iterable)

In [10]:
func = compose_with_reduce([math.exp, math.sin, lambda x: x])
func

<function __main__.compose_with_reduce.<locals>.<lambda>.<locals>.<lambda>(el)>

In [12]:
func(1), math.exp(math.sin(1))

(2.319776824715853, 2.319776824715853)

## 2

In [13]:
import pandas as pd
import numpy as np


In [14]:
df = pd.read_csv("data/ncr_ride_bookings.csv")
df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [17]:
def generate_first3rows(df):
    for i in range(3):
        yield df.iloc[i]

In [18]:
new_df = pd.DataFrame(generate_first3rows(df))
new_df

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card


## 3

In [21]:
import pathway as pw
from typing import Generator, List, Literal, Dict, Optional
import time

In [22]:
def scrape_articles(
    website_urls: list[str],
    refresh_interval: int
) -> Generator:

    indexed_articles: set[str] = set() # set of already scraped articles

    while True: # the content will be scraped until the script is stopped
        for website in website_urls:
            articles = list_articles(website) # Obtaining the articles from the website
            for article in articles:
                if article in indexed_article: # Ignore already scraped articles
                    continue
                
                ... # Compute the URL, the text, and the metadata
                
                indexed_articles.add(article) # add the article to the list to avoid scraping it again
                yield {"url": url, "text": text, "metadata": dict(metadata)} # yield the scraped content

        time.sleep(refresh_interval) # restart the scraping after the refresh interval


In [23]:
from pathway.io.python import ConnectorSubject

class NewsScraperSubject(ConnectorSubject):
    _website_urls: list[str]
    _refresh_interval: int

    def __init__(self, website_urls: list[str], refresh_interval: int) -> None:
        super().__init__()
        self._website_urls = website_urls
        self._refresh_interval = refresh_interval

    def run(self) -> None:
        for article in scrape_articles(
            self._website_urls,
            refresh_interval=self._refresh_interval,
        ):
            url = article["url"]
            text = article["text"]
            metadata = article["metadata"]

            # give the keyword arguments according to the pw.Schema we will define
            # in each iteration, the new article will be appended to the table
            self.next(url=url, text=text, metadata=metadata)


In [25]:
websites = ["https://www.bbc.com/"]

# url should be unique
class ConnectorSchema(pw.Schema):
    url: str = pw.column_definition(primary_key=True)
    text: str
    metadata: dict

subject = NewsScraperSubject(
    website_urls=websites,
    refresh_interval=25,  # if you are encountering API throttling, increase this value, default is 600
)



In [ ]:
pw.run()


Output()